In [1]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset


ModuleNotFoundError: No module named 'mindspore'

下载导入数据集

In [9]:
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download( url, "../data", kind = 'zip', replace = True )


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:03<00:00, 3.29MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ../data


网站给出的训练集目录结构
MNIST_Data
└── train
    ├── train-images-idx3-ubyte (60000个训练图片)
    ├── train-labels-idx1-ubyte (60000个训练标签)
└── test
    ├── t10k-images-idx3-ubyte (10000个测试图片)
    ├── t10k-labels-idx1-ubyte (10000个测试标签)


划分训练集、测试集 -> 化为数据集对象

In [8]:
train_dataset = MnistDataset( 'MNIST_Data/train' )
test_dataset = MnistDataset( 'MNIST_Data/test' )

train_dataset.get_col_names()

['image', 'label']

In [ ]:
def datapip( dataset, batch_size ):
      image_transforms = [
		vision.Rescale( 1.0 / 225.0, 0 ),		# 将输入的图像缩放为1/225
        vision.Normalize( mean = ( 0.1307, ), std = ( 0.3081, ) ),		# 均值和标准差值的归一化处理
        vision.HWC2CHW()
	  ]
      label_transform = transforms.TypeCast( mindspore.int32 )
    
      dataset = dataset.map( image_transforms, 'image' )
      dataset = dataset.map( label_transform, 'label' )
      dataset = dataset.batch( batch_size )
    
      return dataset

In [ ]:
train_dataset = datapip( train_dataset, 64 )
test_dataset = datapip( test_dataset, 64 )

In [ ]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break
for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


In [ ]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)


In [ ]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

# 1. Define forward function
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

# 2. Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# 3. Define function of one-step training
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
